蒙特卡罗模拟

In [102]:
import numpy as np
import pandas as pd
import time
import networkx as nx
import matplotlib.pyplot as plt
from itertools import islice
import random
from scipy.optimize import minimize
import os
print(os.getcwd())
from datetime import datetime
current_time = datetime.now().strftime("%m%d%H%M")
print(current_time)


f:\------------------OWN\VSCode\Python\Math\MathClub\51club\B
05021702


In [91]:
# 读取文件 生成交通需求
def read_excel_data(filepath):

    demands = {}
    demands_dataframe = pd.read_excel(filepath,sheet_name='Demands')
    for index, row in demands_dataframe.iterrows():
        origin_destination = (row['Start'], row['End'])
        demand_value = row['Demand']
        demands[origin_destination] = demand_value * 1.0
    return demands

filepath = r'data/TDNetwork2.xlsx'

In [92]:
#定义并获取交通网络图
def create_graph():
    G = nx.Graph()
    edges = [(1, 2), (1, 4), 
             (2, 3), (2, 5),
             (3, 6),
             (4, 5), (4, 7),
             (5, 6), (5, 8),
             (6, 9),
             (7, 8),
             (8, 9)]
    G.add_edges_from(edges)
    return G
G = create_graph()

In [93]:
def generate_all_paths(G, source, target, cutoff=None):
    """ 生成从source到target的所有简单路径 """
    return list(nx.all_simple_paths(G, source=source, target=target, cutoff=cutoff))

In [94]:
def simulate_network(G, source_target_demands, num_simulations=1000):
    success_rates = []

    for _ in range(num_simulations):
        successful_demand = 0
        total_network_demand = sum(source_target_demands.values())  # 网络总需求

        for (source, target), demand in source_target_demands.items():
            paths = generate_all_paths(G, source, target)  # 获取k最短路径
            path_demands = np.random.randint(1, demand, size=len(paths))  # 随机分配
            path_demands = path_demands / path_demands.sum() * demand  # 归一化以匹配该起止点的总需求

            for path, path_demand in zip(paths, path_demands):
                if all(random.random() >= 1/12 for _ in path[:-1]):  # 检查每条路径上的边是否未中断
                    successful_demand += path_demand

        success_rate = successful_demand / total_network_demand if total_network_demand > 0 else 0
        success_rates.append(success_rate)

    return np.mean(success_rates) * 100  # 返回平均成功率的百分比形式


In [95]:
# 运行模拟
demands = read_excel_data(filepath)
average_success_rate = simulate_network(G, demands)
print(f"Average success rate: {average_success_rate * 100:.2f}%")

Average success rate: 6625.43%
